<a href="https://colab.research.google.com/github/Bibhash123/Resizing_Forensics/blob/main/proposed%20(preprocessing%20layer)/Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Uncompressed Images Dataset

In [ ]:
.from IPython.display import clear_output
!pip install -q kaggle
!mkdir ~/.kaggle/
!cp kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d mrutyunjaybiswal/raisetiff-uncompressed-images-dataset-x300
!unzip raisetiff-uncompressed-images-dataset-x300.zip -d '/content/UC/'
clear_output(wait=False)

# Necessary Libraries

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K

# Create Model

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
class PreprocessLayer(L.Layer):
  def __init__(self,**kwargs):
    super(PreprocessLayer,self).__init__(**kwargs)
  
  def build(self,input_shape):
    self.built=True
  #filter definitions
  #first third and second defined....to define sq and edge
  #elementary filters  
  def a1(self,image):
    filter = K.constant(np.array([[[[0,0,0,0,0],
                        [0,0,0,0,0],
                        [0,0,-1,1,0],
                        [0,0,0,0,0],
                        [0,0,0,0,0]]]]).reshape((5,5,1,1)))
    return K.conv2d(image,filter,padding='same')
  
  def lin_vertical(self,image):
    filter = K.constant(np.array([[[[0,0,0,0,0],
                        [0,0,1,0,0],
                        [0,0,-1,0,0],
                        [0,0,0,0,0],
                        [0,0,0,0,0]]]]).reshape((5,5,1,1)))
    return K.conv2d(image,filter,padding='same')
  
  def lin_horizontal(self,image):
    filter = K.constant(np.array([[[[0,0,0,0,0],
                        [0,0,0,0,0],
                        [0,1,-1,0,0],
                        [0,0,0,0,0],
                        [0,0,0,0,0]]]]).reshape((5,5,1,1)))
    return K.conv2d(image,filter,padding='same')
  
  def a2(self,image):
    filter = K.constant(np.array([[[[0,0,0,0,0],
                        [0,0,0,0,0],
                        [0,1,-2,1,0],
                        [0,0,0,0,0],
                        [0,0,0,0,0]]]]).reshape((5,5,1,1)))
    return K.conv2d(image,filter,padding='same')
  
  def ord2_vertical(self,image):
    filter = K.constant(np.array([[[[0,0,0,0,0],
                        [0,0,1,0,0],
                        [0,0,-2,0,0],
                        [0,0,1,0,0],
                        [0,0,0,0,0]]]]).reshape((5,5,1,1)))
    return K.conv2d(image,filter,padding='same')

  def a3(self,image):
    filter = K.constant(np.array([[[[0,0,0,0,0],
                        [0,0,0,0,0],
                        [0,1,-3,3,-1],
                        [0,0,0,0,0],
                        [0,0,0,0,0]]]]).reshape((5,5,1,1)))
    return K.conv2d(image,filter,padding='same')
  
  def ord3_horizontal(self,image):
    filter = K.constant(np.array([[[[0,0,0,0,0],
                        [0,0,0,0,0],
                        [-1,3,-3,1,0],
                        [0,0,0,0,0],
                        [0,0,0,0,0]]]]).reshape((5,5,1,1)))
    return K.conv2d(image,filter,padding='same')
  
  def ord3_vertical(self,image):
    filter = K.constant(np.array([[[[0,0,-1,0,0],
                        [0,0,3,0,0],
                        [0,0,-3,0,0],
                        [0,0,1,0,0],
                        [0,0,0,0,0]]]]).reshape((5,5,1,1)))
    return K.conv2d(image,filter,padding='same')

  #minmax filters
  def b1(self,image):
    return K.minimum(self.a1(image),self.lin_horizontal(image))
  
  def c1(self,image):
    return K.minimum(self.a1(image),self.lin_vertical(image))
  
  def d1(self,image):
    return K.minimum(self.c1(image),self.lin_horizontal(image))
  
  def b2(self,image):
    return K.minimum(self.a2(image),self.ord2_vertical(image))
  
  def b3(self,image):
    return K.minimum(self.a3(image),self.ord3_horizontal(image))
  
  def c3(self,image):
    return K.minimum(self.a3(image),self.ord3_vertical(image))
  
  def d3(self,image):
    return K.minimum(self.c3(image),self.ord3_horizontal(image))
  
  #square filters
  def s3a(self,image):
    filter = K.constant(np.array([[[[0,0,0,0,0],
                        [0,-1,2,-1,0],
                        [0,2,-4,2,0],
                        [0,-1,2,-1,0],
                        [0,0,0,0,0]]]]).reshape((5,5,1,1)))
    return K.conv2d(image,filter,padding='same')
  
  def s5a(self,image):
    filter = K.constant(np.array([[[[-1,2,-2,2,-1],
                        [2,-6,8,-6,2],
                        [-2,8,-12,8,-2],
                        [2,-6,8,-6,2],
                        [-1,2,-2,2,-1]]]]).reshape((5,5,1,1)))
    return K.conv2d(image,filter,padding='same')
  
  #edge filters
  def e3_vertical_l(self,image):
    filter = K.constant(np.array([[[[0,0,0,0,0],
                        [0,-1,2,0,0],
                        [0,2,-4,0,0],
                        [0,-1,2,0,0],
                        [0,0,0,0,0]]]]).reshape((5,5,1,1)))
    return K.conv2d(image,filter,padding='same')
  
  def e3_horizontal(self,image):
    filter = K.constant(np.array([[[[0,0,0,0,0],
                        [0,0,0,0,0],
                        [0,2,-4,2,0],
                        [0,-1,2,-1,0],
                        [0,0,0,0,0]]]]).reshape((5,5,1,1)))
    return K.conv2d(image,filter,padding='same')

  def e3_vertical_r(self,image):
    filter = K.constant(np.array([[[[0,0,0,0,0],
                        [0,0,2,-1,0],
                        [0,0,-4,2,0],
                        [0,0,2,-1,0],
                        [0,0,0,0,0]]]]).reshape((5,5,1,1)))
    return K.conv2d(image,filter,padding='same')
  
  def e3a(self,image):
    filter = K.constant(np.array([[[[0,0,0,0,0],
                        [0,-1,2,-1,0],
                        [0,2,-4,2,0],
                        [0,0,0,0,0],
                        [0,0,0,0,0]]]]).reshape((5,5,1,1)))
    return K.conv2d(image,filter,padding='same')
  
  def e3b(self,image):
    return K.minimum(self.e3a(image),self.e3_vertical_l(image))
  
  def e3c(self,image):
    return K.minimum(self.e3a(image),self.e3_horizontal(image))
  
  def e3d(self,image):
    return K.minimum(self.e3c(image),K.minimum(self.e3_vertical_l(image),self.e3_vertical_r(image)))
  
  def e5_vertical_l(self,image):
    filter = K.constant(np.array([[[[-1,2,-2,0,0],
                        [2,-6,8,0,0],
                        [-2,8,-12,0,0],
                        [2,-6,8,0,0],
                        [-1,2,-2,0,0]]]]).reshape((5,5,1,1)))
    return K.conv2d(image,filter,padding='same')
  
  def e5_vertical_r(self,image):
    filter = K.constant(np.array([[[[0,0,-2,2,-1],
                        [0,0,8,-6,2],
                        [0,0,-12,8,-2],
                        [0,0,8,-6,2],
                        [0,0,-2,2,-1]]]]).reshape((5,5,1,1)))
    return K.conv2d(image,filter,padding='same')
  
  def e5_horizontal(self,image):
    filter = K.constant(np.array([[[[0,0,0,0,0],
                        [0,0,0,0,0],
                        [-2,8,-12,8,-2],
                        [2,-6,8,-6,2],
                        [-1,2,-2,2,-1]]]]).reshape((5,5,1,1)))
    return K.conv2d(image,filter,padding='same')
  
  def e5a(self,image):
    filter = K.constant(np.array([[[[-1,2,-2,2,-1],
                        [2,-6,8,-6,2],
                        [-2,8,-12,8,-2],
                        [0,0,0,0,0],
                        [0,0,0,0,0]]]]).reshape((5,5,1,1)))
    return K.conv2d(image,filter,padding='same')
  
  def e5b(self,image):
    return K.minimum(self.e5a(image),self.e5_vertical_l(image))
  
  def e5c(self,image):
    return K.minimum(self.e5a(image),self.e5_horizontal(image))
  
  def e5d(self,image):
    return K.minimum(self.e5c(image),K.minimum(self.e5_vertical_l(image),self.e5_vertical_r(image)))

  def call(self,image):
    return K.stack([self.a1(image)[:,:,:,0],
                                self.b1(image)[:,:,:,0],
                                self.c1(image)[:,:,:,0],
                                self.d1(image)[:,:,:,0],
                                self.a2(image)[:,:,:,0],
                                self.b2(image)[:,:,:,0],
                                self.a3(image)[:,:,:,0],
                                self.b3(image)[:,:,:,0],
                                self.c3(image)[:,:,:,0],
                                self.d3(image)[:,:,:,0],
                                self.s3a(image)[:,:,:,0],
                                self.s5a(image)[:,:,:,0],
                                self.e3a(image)[:,:,:,0],
                                self.e3b(image)[:,:,:,0],
                                self.e3c(image)[:,:,:,0],
                                self.e3d(image)[:,:,:,0],
                                self.e5a(image)[:,:,:,0],
                                self.e5b(image)[:,:,:,0],
                                self.e5c(image)[:,:,:,0],
                                self.e5d(image)[:,:,:,0],
                                ],axis=3)
  def get_output_shape(self,input_shape):
    return (input_shape[0],input_shape[1],input_shape[2],20)

In [ ]:
class ConvConst(tf.keras.models.Model):
  def __init__(self,num_classes=10):
    super(ConvConst,self).__init__()
    self.n_classes = num_classes
    self.block1 = PreprocessLayer()
    
    self.block2 = tf.keras.models.Sequential([
                                              L.Conv2D(96,(7,7),strides=2),
                                              L.BatchNormalization(),
                                              L.Activation('tanh'),
                                              L.MaxPool2D(pool_size=(3,3),strides=2)
    ])
    self.block3 = tf.keras.models.Sequential([
                                             L.Conv2D(64,(5,5),strides=1),
                                             L.BatchNormalization()
    ])
    self.block4 = tf.keras.models.Sequential([
                                             L.Conv2D(64,(5,5),strides=1),
                                             L.BatchNormalization(),
                                             L.Activation('tanh'),
                                             L.MaxPool2D(pool_size=(3,3),strides=2)
    ])
    self.block5 = tf.keras.models.Sequential([
                                             L.Conv2D(128,(1,1),strides=1),
                                             L.BatchNormalization(),
                                             L.Activation('tanh'),
                                             L.AveragePooling2D(pool_size=(3,3),strides=2)
    ])

    self.fc1 = L.Dense(200)
    self.fc2 = L.Dense(200)
    self.fc3 = L.Dense(num_classes,activation="softmax")

  def call(self,inputs,**kwargs):
    out = self.block1(inputs)
    out = self.block2(out)
    out = self.block3(out)
    out = self.block4(out)
    out = self.block5(out)

    out = L.Flatten()(out)

    out = self.fc1(out)
    out = self.fc2(out)
    out = self.fc3(out)
    return out

In [ ]:
K.clear_session()
model = ConvConst(num_classes=10)
model.compile(loss = 'sparse_categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),metrics=["accuracy"])
model.build((None,256,256,1))

model.layers[0].trainable=False

# Load Saved Weights

In [ ]:
model.load_weights('model_1_3.hdf5')

# Produce Results

In [ ]:
import glob
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score
from utilities import PrepData

## Accuracy Varying with resizing factor

In [ ]:
QF1 = [50,60,70,80,90]
QF2 = [90]
RF = [0.6,0.7,0.8,0.9,0.95,1.05,1.1,1.2,1.3,1.4]

Prepare a dataset of double compressed images. The PrepData algorithm can be found in utilities.py

In [ ]:
PrepData(path='/content/UC/test_tiff/',QF1= QF1, QF2= QF2, RF= RF)

In [ ]:
path = '/content/dcr/Test/'
factor = lambda x: x.split('.jpg')[0].split('rf_')[-1]

In [ ]:
def softmax(arr):
  return np.exp(arr)/np.sum(np.exp(arr))
def crop_image(img):
  mid = (int(img.shape[0]/2),int(img.shape[1]/2))
  return img[mid[0]-128:mid[0]+128,mid[1]-128:mid[1]+128]
def rf_acc(path,rf):
  #get accuracy for given resizing factor
  count=0
  Y_true = []
  Y_pred = []
  images = []
  for img in tqdm(os.listdir(path)):
    if float(factor(img))==rf:
      count+=1
      image = cv2.cvtColor(cv2.imread(os.path.join(path,img)),cv2.COLOR_BGR2GRAY)
      image = np.expand_dims(crop_image(image),axis=2).astype(np.float32)
      images.append(image)
      Y_true.append(RF.index(rf))
  preds = np.stack([np.argmax(softmax(p)) for p in model.predict(np.stack(images))])
  print(f'Images: {count}')
  return accuracy_score(Y_true,preds)

In [ ]:
rf_acc(path,1.4)


Images: 5380


0.9351301115241636

In [ ]:
rf_acc(path,1.3)


Images: 5380


0.9775092936802974

In [ ]:
rf_acc(path,1.2)


Images: 5380


0.9353159851301115

In [ ]:
rf_acc(path,1.1)


Images: 5380


0.962639405204461

In [ ]:
rf_acc(path,1.05)


Images: 5380


0.9433085501858736

In [ ]:
rf_acc(path,0.95)


Images: 5380


0.9065055762081784

In [ ]:
rf_acc(path,0.9)


Images: 5380


0.8901486988847583

In [ ]:
rf_acc(path,0.8)


Images: 5380


0.9295539033457249

In [ ]:
rf_acc(path,0.7)


Images: 5380


0.8511152416356877

In [ ]:
rf_acc(path,0.6)


Images: 5380


0.858550185873606

## Accuracy Varying with Quality Factor 2

In [ ]:
QF1 = [50,60,70,80,90]
QF2 = [50,60,70,80,90,99]
RF = [0.6,0.7,0.8,0.9,0.95,1.05,1.1,1.2,1.3,1.4]

In [ ]:
PrepData(path='/content/UC/test_tiff/',QF1= QF1, QF2= QF2, RF= RF)



Performing Double Compression with tampering:


In [ ]:
q_factor = lambda x: int(x.split('_rf_')[0].split('_Q2_')[1])
q1_factor = lambda x: int(x.split('_Q2_')[0].split('_Q1_')[1])
def qf_acc(path,qf):
  accuracies = []
  for qf1 in tqdm(QF1):
    count=0
    r_counts = []
    for rf in RF:
      r_count = 0
      Y_true = []
      Y_pred = []
      images = []
      for img in tqdm(os.listdir(path),leave=False):
        if q_factor(img)==qf and q1_factor(img)==qf1 and float(factor(img))==rf:
          count+=1
          r_count+=1
          image = cv2.cvtColor(cv2.imread(os.path.join(path,img)),cv2.COLOR_BGR2GRAY)
          image = np.expand_dims(crop_image(image),axis=2).astype(np.float32)
          images.append(image)
          Y_true.append(RF.index(rf))
      preds = np.stack([np.argmax(softmax(p)) for p in model.predict(np.stack(images))])
      accuracies.append(accuracy_score(Y_true,preds))
      r_counts.append(r_count)

    print(f'Images for {qf1}: {count}')
    print(dict(zip(RF,r_counts)))
      
  return np.mean(accuracies)

In [ ]:
qf_acc(path,50)

Images for 50: 2049
{0.6: 212, 0.7: 211, 0.8: 209, 0.9: 231, 0.95: 197, 1.05: 212, 1.1: 183, 1.2: 190, 1.3: 181, 1.4: 223}


Images for 60: 2061
{0.6: 216, 0.7: 197, 0.8: 200, 0.9: 232, 0.95: 186, 1.05: 191, 1.1: 197, 1.2: 217, 1.3: 223, 1.4: 202}


Images for 70: 2046
{0.6: 198, 0.7: 200, 0.8: 219, 0.9: 186, 0.95: 204, 1.05: 199, 1.1: 221, 1.2: 199, 1.3: 214, 1.4: 206}


Images for 80: 2059
{0.6: 220, 0.7: 192, 0.8: 208, 0.9: 194, 0.95: 191, 1.05: 224, 1.1: 241, 1.2: 212, 1.3: 188, 1.4: 189}


Images for 90: 2051
{0.6: 227, 0.7: 204, 0.8: 211, 0.9: 190, 0.95: 202, 1.05: 200, 1.1: 194, 1.2: 222, 1.3: 219, 1.4: 182}



0.705383826869832

In [ ]:
qf_acc(path,60)

Images for 50: 2052
{0.6: 215, 0.7: 214, 0.8: 214, 0.9: 215, 0.95: 217, 1.05: 194, 1.1: 202, 1.2: 190, 1.3: 194, 1.4: 197}


Images for 60: 2036
{0.6: 210, 0.7: 191, 0.8: 188, 0.9: 213, 0.95: 224, 1.05: 196, 1.1: 204, 1.2: 199, 1.3: 211, 1.4: 200}


Images for 70: 2043
{0.6: 189, 0.7: 237, 0.8: 197, 0.9: 197, 0.95: 234, 1.05: 183, 1.1: 205, 1.2: 200, 1.3: 210, 1.4: 191}


Images for 80: 2054
{0.6: 196, 0.7: 200, 0.8: 209, 0.9: 221, 0.95: 221, 1.05: 209, 1.1: 190, 1.2: 217, 1.3: 183, 1.4: 208}


Images for 90: 2038
{0.6: 234, 0.7: 202, 0.8: 204, 0.9: 214, 0.95: 194, 1.05: 183, 1.1: 187, 1.2: 197, 1.3: 190, 1.4: 233}



0.755665812451374

In [ ]:
qf_acc(path,70)

Images for 50: 2034
{0.6: 196, 0.7: 226, 0.8: 201, 0.9: 202, 0.95: 216, 1.05: 204, 1.1: 214, 1.2: 202, 1.3: 178, 1.4: 195}


Images for 60: 2053
{0.6: 177, 0.7: 196, 0.8: 208, 0.9: 210, 0.95: 176, 1.05: 216, 1.1: 235, 1.2: 207, 1.3: 212, 1.4: 216}


Images for 70: 2045
{0.6: 182, 0.7: 202, 0.8: 202, 0.9: 207, 0.95: 201, 1.05: 195, 1.1: 222, 1.2: 203, 1.3: 215, 1.4: 216}


Images for 80: 2048
{0.6: 232, 0.7: 201, 0.8: 216, 0.9: 194, 0.95: 193, 1.05: 203, 1.1: 191, 1.2: 202, 1.3: 210, 1.4: 206}


Images for 90: 2043
{0.6: 217, 0.7: 208, 0.8: 213, 0.9: 184, 0.95: 194, 1.05: 202, 1.1: 223, 1.2: 210, 1.3: 185, 1.4: 207}



0.8150600303506791

In [ ]:
qf_acc(path,80)

Images for 50: 2049
{0.6: 194, 0.7: 207, 0.8: 203, 0.9: 225, 0.95: 227, 1.05: 215, 1.1: 202, 1.2: 191, 1.3: 182, 1.4: 203}


Images for 60: 2056
{0.6: 189, 0.7: 194, 0.8: 214, 0.9: 214, 0.95: 214, 1.05: 206, 1.1: 180, 1.2: 225, 1.3: 221, 1.4: 199}


Images for 70: 2044
{0.6: 231, 0.7: 180, 0.8: 230, 0.9: 200, 0.95: 202, 1.05: 197, 1.1: 214, 1.2: 200, 1.3: 190, 1.4: 200}


Images for 80: 2032
{0.6: 197, 0.7: 207, 0.8: 191, 0.9: 202, 0.95: 182, 1.05: 214, 1.1: 206, 1.2: 217, 1.3: 214, 1.4: 202}


Images for 90: 2051
{0.6: 204, 0.7: 203, 0.8: 196, 0.9: 203, 0.95: 196, 1.05: 182, 1.1: 202, 1.2: 224, 1.3: 222, 1.4: 219}



0.8715347637671704

In [ ]:
qf_acc(path,90)

Images for 50: 2043
{0.6: 224, 0.7: 213, 0.8: 197, 0.9: 205, 0.95: 198, 1.05: 205, 1.1: 207, 1.2: 204, 1.3: 213, 1.4: 177}


Images for 60: 2035
{0.6: 233, 0.7: 192, 0.8: 213, 0.9: 220, 0.95: 199, 1.05: 195, 1.1: 197, 1.2: 209, 1.3: 176, 1.4: 201}


Images for 70: 2033
{0.6: 192, 0.7: 215, 0.8: 200, 0.9: 186, 0.95: 221, 1.05: 214, 1.1: 199, 1.2: 185, 1.3: 203, 1.4: 218}


Images for 80: 2043
{0.6: 191, 0.7: 195, 0.8: 226, 0.9: 210, 0.95: 222, 1.05: 195, 1.1: 185, 1.2: 204, 1.3: 198, 1.4: 217}


Images for 90: 2035
{0.6: 206, 0.7: 209, 0.8: 193, 0.9: 201, 0.95: 208, 1.05: 180, 1.1: 218, 1.2: 183, 1.3: 225, 1.4: 212}



0.9196580275981006

In [ ]:
qf_acc(path,99)

Images for 50: 2050
{0.6: 191, 0.7: 206, 0.8: 208, 0.9: 222, 0.95: 226, 1.05: 196, 1.1: 197, 1.2: 200, 1.3: 183, 1.4: 221}


Images for 60: 2040
{0.6: 193, 0.7: 220, 0.8: 204, 0.9: 211, 0.95: 199, 1.05: 202, 1.1: 203, 1.2: 196, 1.3: 201, 1.4: 211}


Images for 70: 2041
{0.6: 217, 0.7: 190, 0.8: 213, 0.9: 209, 0.95: 183, 1.05: 194, 1.1: 237, 1.2: 192, 1.3: 194, 1.4: 212}


Images for 80: 2054
{0.6: 202, 0.7: 223, 0.8: 210, 0.9: 192, 0.95: 171, 1.05: 213, 1.1: 233, 1.2: 207, 1.3: 205, 1.4: 198}


Images for 90: 2044
{0.6: 219, 0.7: 177, 0.8: 210, 0.9: 211, 0.95: 221, 1.05: 201, 1.1: 191, 1.2: 192, 1.3: 208, 1.4: 214}



0.9447716711731742